In [4]:
import pandas as pd

## I had to download lxml, html5lib and BeautifulSoup4 to extract wikipedia table 

In [4]:
pip install lxml

     |████████████████████████████████| 5.8MB 3.9MB/s eta 0:00:01     |██▊                             | 481kB 3.9MB/s eta 0:00:02
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 10.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page,  attrs={"class":"wikitable"})

print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [2]:
wikitables[0]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [5]:
df_Canada = pd.DataFrame(wikitables[0])
df_Canada.shape

(287, 3)

In [7]:
df_Canada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
df_Canada

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


## I have used .drop pandas class for me to eliminate 'not assigned' rows 

In [19]:
df_Canada.drop(df_Canada[ df_Canada['Neighbourhood'] == 'Not assigned' ].index, inplace=True)

In [33]:
df_Canada[0:15]

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [34]:
df_Canada.shape

(210, 3)

In [35]:
geo = pd.read_csv("Geospatial_Coordinates.csv") 
geo.rename(index=str, columns={'Postal Code':'Postcode'}, inplace=True)
geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
neigh = pd.merge(df_Canada, geo, how='outer', on=['Postcode'])
neigh.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188


# Explore and cluster the neighborhoods in Toronto

In [52]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [37]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neigh['Borough'].unique()),
        neigh.shape[0]
    )
)

The dataframe has 10 boroughs and 210 neighborhoods.


In [40]:
North_York = neigh[neigh['Borough'] == 'North York'].reset_index(drop=True)
North_York.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,Lawrence Heights,43.718518,-79.464763
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M3B,North York,Don Mills North,43.745906,-79.352188


In [46]:
CLIENT_ID = '11BKQM0UGVYEPCREYES3GUJMWMUGNOITTXFLFW10RIENRXG0' # your Foursquare ID
CLIENT_SECRET = 'E4F4EDZXYBUTZ3XZ0FZQ0AL4KKO01BUQJZ3MKBXIENLIARCY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 11BKQM0UGVYEPCREYES3GUJMWMUGNOITTXFLFW10RIENRXG0
CLIENT_SECRET:E4F4EDZXYBUTZ3XZ0FZQ0AL4KKO01BUQJZ3MKBXIENLIARCY


In [47]:
North_York.loc[0, 'Neighbourhood']

'Parkwoods'

In [48]:
neighborhood_latitude = North_York.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_York.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_York.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [49]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=11BKQM0UGVYEPCREYES3GUJMWMUGNOITTXFLFW10RIENRXG0&client_secret=E4F4EDZXYBUTZ3XZ0FZQ0AL4KKO01BUQJZ3MKBXIENLIARCY&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e56e270b57e88001b457bcc'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA'

In [54]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [55]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741
3,Corrosion Service Company Limited,Construction & Landscaping,43.752432,-79.334661


# Explore Neighborhoods in North York

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
North_York_venues = getNearbyVenues(names=North_York['Neighbourhood'],
                                   latitudes=North_York['Latitude'],
                                   longitudes=North_York['Longitude']
                                  )


Parkwoods
Victoria Village
Lawrence Heights
Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park
Don Mills South
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Fairview
Henry Farm
Oriole
Northwood Park
York University
Bayview Village
CFB Toronto
Downsview East
Silver Hills
York Mills
Downsview West
Downsview
North Park
Upwood Park
Humber Summit
Newtonbrook
Willowdale
Downsview Central
Bedford Park
Lawrence Manor East
Emery
Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [62]:
print(North_York.shape)
North_York.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,Lawrence Heights,43.718518,-79.464763
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M3B,North York,Don Mills North,43.745906,-79.352188


In [63]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bathurst Manor,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park,23,23,23,23,23,23
CFB Toronto,4,4,4,4,4,4
Don Mills North,4,4,4,4,4,4
Don Mills South,22,22,22,22,22,22
Downsview,4,4,4,4,4,4
Downsview Central,2,2,2,2,2,2
Downsview East,4,4,4,4,4,4


In [64]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 106 uniques categories.


In [68]:
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
North_York_onehot.shape

(483, 107)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [73]:
North_York_onehot = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bathurst Manor,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.05,0.0,0.05,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.05,0.050000,0.050000,0.00,0.0,0.0,0.0,0.000000,0.0,0.050000,0.0,0.050000,0.0,0.05,0.00,0.0,0.0,0.05,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.0,0.0
1,Bayview Village,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.00,0.25,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,Bedford Park,0.0,0.00,0.043478,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.043478,0.00,0.00,0.0,0.0,0.086957,0.043478,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.043478,0.043478,0.0,0.00,0.0,0.0,0.0,0.00,0.043478,0.0,0.0,0.086957,0.00,0.043478,0.043478,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.043478,0.043478,0.00,0.0,0.0,0.0,0.043478,0.0,0.086957,0.0,0.086957,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.043478,0.0,0.043478,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,CFB Toronto,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.00,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.25,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Don Mills North,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.25,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0

In [74]:
North_York_onehot.shape

(36, 107)

## Let's print each neighborhood along with the top 5 most common venues

In [75]:
num_top_venues = 5

for hood in North_York_onehot['Neighborhood']:
    print("----"+hood+"----")
    temp = North_York_onehot[North_York_onehot['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor----
                venue  freq
0         Coffee Shop  0.10
1               Diner  0.05
2  Frozen Yogurt Shop  0.05
3      Sandwich Place  0.05
4       Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3          Restaurant  0.09
4    Greek Restaurant  0.04


----CFB Toronto----
          venue  freq
0    Playground  0.25
1   Snack Place  0.25
2          Park  0.25
3       Airport  0.25
4  Hockey Arena  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4     Accessories Store  0.00


----Don Mills South----
              venue  freq
0     

## Cluster Neighborhoods

In [78]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bathurst Manor,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.05,0.0,0.05,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.05,0.050000,0.050000,0.00,0.0,0.0,0.0,0.000000,0.0,0.050000,0.0,0.050000,0.0,0.05,0.00,0.0,0.0,0.05,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.0,0.0
1,Bayview Village,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.00,0.25,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,Bedford Park,0.0,0.00,0.043478,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.043478,0.00,0.00,0.0,0.0,0.086957,0.043478,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.043478,0.043478,0.0,0.00,0.0,0.0,0.0,0.00,0.043478,0.0,0.0,0.086957,0.00,0.043478,0.043478,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.043478,0.043478,0.00,0.0,0.0,0.0,0.043478,0.0,0.086957,0.0,0.086957,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.043478,0.0,0.043478,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,CFB Toronto,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.00,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.25,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Don Mills North,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.25,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0

## Run k-means to cluster the neighborhood into 5 clusters.

In [79]:
# set number of clusters
kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 3, 1, 0, 0], dtype=int32)

## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [84]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor,Coffee Shop,Diner,Supermarket,Bridal Shop,Pizza Place,Restaurant,Pharmacy,Sandwich Place,Pet Store,Shopping Mall
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Distribution Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Bedford Park,Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Liquor Store,Café,Juice Bar,Butcher,Pharmacy,Pizza Place
3,CFB Toronto,Snack Place,Airport,Playground,Park,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
4,Don Mills North,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Distribution Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [87]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_York_merged = North_York

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

North_York_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [88]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighbourhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'latitude' is not defined